In [86]:
import pandas as pd

In [99]:
df = pd.read_csv("/Users/denisovamaria/PycharmProjects/evroopt_data_preparing/evroopt_data.csv")

In [88]:
df

,Unnamed: 0,name,price,category
0,0,"Капуста белокочанная, 1 кг",1.35,Овощи
1,1,"Картофель продовольственный, мытый, фасованный...",2.79,Овощи
2,2,Картофель мытый,2.49,Овощи
3,3,"Огурец гладкий, 1 кг",8.09,Овощи
4,4,"Картофель, 1 кг",1.10,Овощи
...,...,...,...,...
10086,10952,"Лопата для снега «Инструм-Агро» Снежок, планка...",16.08,Автотовары
10087,10953,"Антифриз «Onzoil» Optimal, G 11, 900 мл",9.95,Автотовары
10088,10954,"Буксировочный трос «Felix» 5 т, 5 м",23.05,Автотовары
10089,10955,"Пуско-зарядное устройство «AVS» Energy BC-400,...",37.99,Автотовары


In [89]:
df['category'] = df['category'].astype(str)


In [90]:
def preprocess_text(text):
    text = text.lower()
    return text

## класторищаци dbscan

In [91]:
from sklearn.feature_extraction.text import TfidfVectorizer

shop_categories = list(df["category"])
shop_categories_processed = df["name"] + " " + [preprocess_text(cat) for cat in shop_categories]

# Преобразование текста в векторы
vectorizer = TfidfVectorizer()
shop_vectors = vectorizer.fit_transform(shop_categories_processed)


In [92]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.5, min_samples=2, metric='cosine')
labels = dbscan.fit_predict(shop_vectors)

df['cluster'] = labels

# Печать результатов
df[['name', 'category', 'cluster']].sort_values(by='cluster')

,name,category,cluster
1807,"Изделия булочные «Рулетики» с маком, 250 г",Выпечка,-1
3456,"Набор чая «Curtis» Dessert Tea Collection, 58.5 г",Чай,-1
3466,"Набор чая «Richard» Royl Selection, ассорти, 1...",Чай,-1
3505,"Чай красный «Dalai» very berry, 25х2 г",Чай,-1
3515,"Чай черный «TeaBerry» Ночь Клеопатры, 100 г",Чай,-1
...,...,...,...
10072,"Пластилин цветной «Гамма» Мультики, 280018, 12...",Товары для творчества,532
10081,"Смазка «Onzoil» XX-40, 210 мл",Автотовары,533
10082,"Смазка силиконовая «Onzoil» XX-Si, 210 мл",Автотовары,533
10089,"Пуско-зарядное устройство «AVS» Energy BC-400,...",Автотовары,534


In [93]:
df.loc[df["cluster"] == -1].count()

Unnamed: 0    864
name          864
price         864
category      864
cluster       864
dtype: int64

In [94]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.7, min_samples=5, metric='cosine')  # 'cosine' для вычисления сходства между векторами
labels = dbscan.fit_predict(shop_vectors)

df['cluster'] = labels

# Печать результатов
df[['name', 'category', 'cluster']].sort_values(by='cluster')

,name,category,cluster
10090,Пуско-зарядное устройство «AVS» Energy Expert ...,Автотовары,-1
4647,Солод ржаной «Мира» 700 г,Мука,-1
4635,"Солод ржаной «Для кваса и хлеба» сухой, фермен...",Мука,-1
7523,Гель для душа детский «Вiтэкс» Girl Kosmo Magi...,Детская гигиена,-1
7001,Закуска Горошница «Полезные продукты» со вкусо...,Для вегетарианцев,-1
...,...,...,...
10067,Стакан-непроливайка «Гамма» 2305215,Товары для творчества,21
10066,"Набор кистей «ArtSpace» Пони №2,3,5",Товары для творчества,21
10063,"Акварель «Мульти-Пульти» Чебурашка, АПЧБ_53006...",Товары для творчества,21
10061,Бумага цветная двусторонняя «Мульти-Пульти» Че...,Товары для творчества,21


In [95]:
df = df.loc[df["cluster"] != -1].count()

In [96]:
from sklearn.metrics import silhouette_score

score = silhouette_score(shop_vectors, labels, metric='cosine')
print(f'Silhouette Score: {score}')


Silhouette Score: -0.013331180197121264


## обработка категорий

In [97]:
belstat_categories = ['мясо и мясопродукты',
                      'рыба и рыбопродукты',
                      'молоко и молочные продукты',
                      'мороженое',
                      'сыры',
                      'масло и жиры',
                      'яйца куриные',
                      'сахар',
                      'чай',
                      'кофе',
                      'мед натуральный',
                      'соль поваренная пищевая',
                      'мука пшеничная',
                      'хлеб и изделия хлебобулочные',
                      'крупа и бобовые',
                      'макаронные изделия',
                      'кондитерские изделия',
                      'плодоовощная продукция',
                      'напитки безалкогольные',

                      'прочие продовольственные товары',
                      'по имени',

                      'ткани и пряжа',
                      'одежда',
                      'обувь',
                      'мебель',
                      'ковры и ковровые изделия',
                      'посуда, приборы столовые',
                      'электротовары',
                      'телерадиотовары',
                      'строительные материалы',
                      'галантерея',
                      'автомобили легковые и запчасти',
                      'велосипеды и мотоциклы',
                      'печатные издания',
                      'топливо горючесмазочные материалы',
                      'часы наручные',
                      'мыло хозяйственное',
                      'товары бытовой химии',
                      'товары хозяйственные',
                      'ювелирные изделия',

                      'прочие культурнобытовые товары',

                      'товары для спорта и туризма',
                      'письменные товары, канцелярские принадлежности',
                      'товары парфюмерно-косметические',
                      'бумажнобеловые товары',
                      'прочие товары']

In [100]:
shop_categories = []
for i in df['category'].unique():
    shop_categories.append(i)


def preprocess_text(text):
    text = text.lower()
    return text


shop_categories_processed = [preprocess_text(cat) for cat in shop_categories]
belstat_categories_processed = [preprocess_text(cat) for cat in belstat_categories]

In [101]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer()
shop_vectors = vectorizer.fit_transform(shop_categories_processed)
belstat_vectors = vectorizer.transform(belstat_categories_processed)

category_mapping = {}
threshold = 0.4
for i, shop_cat in enumerate(shop_categories_processed):
    similarities = cosine_similarity(shop_vectors[i], belstat_vectors)
    best_match_index = similarities.argmax()
    if similarities.max() < threshold:
        category_mapping[shop_cat] = 'Другое'
    else:
        category_mapping[shop_cat] = belstat_categories[best_match_index]


In [102]:
category_mapping = {'овощи': 'плодоовощная продукция', 'фрукты': 'плодоовощная продукция',
                    'зелень': 'плодоовощная продукция', 'грибы': 'плодоовощная продукция',
                    'ягоды': 'плодоовощная продукция',
                    'квашения, соления, салаты': 'плодоовощная продукция', 'мороженое': 'мороженое',
                    'яйца': 'яйца куриные',
                    'молоко, сливки': 'молоко и молочные продукты',
                    'кефир, кисломолочные изделия': 'молоко и молочные продукты', 'масло, маргарин': 'масло и жиры',
                    'сметана, творог': 'молоко и молочные продукты', 'сыры': 'сыры',
                    'йогурты, десерты, каши': 'молоко и молочные продукты',
                    'хлеб, лепешки, тарталетки': 'хлеб и изделия хлебобулочные',
                    'торты, пирожные': 'кондитерские изделия', 'выпечка': 'кондитерские изделия',
                    'сушки, пряники': 'кондитерские изделия',
                    'замороженный хлеб, выпечка': 'хлеб и изделия хлебобулочные',
                    'свинина': 'мясо и мясопродукты', 'птица': 'мясо и мясопродукты', 'говядина': 'мясо и мясопродукты',
                    'другие виды мяса': 'мясо и мясопродукты',
                    'полуфабрикаты': 'мясо и мясопродукты', 'мясные изделия': 'мясо и мясопродукты',
                    'колбасы копченые, сыровяленые': 'мясо и мясопродукты',
                    'рыба свежемороженая': 'рыба и рыбопродукты',
                    'морепродукты': 'рыба и рыбопродукты', 'рыба готовая': 'рыба и рыбопродукты',
                    'икра': 'рыба и рыбопродукты',
                    'рыба охлажденная': 'рыба и рыбопродукты', 'фрукты, ягоды замороженные': 'плодоовощная продукция',
                    'замороженные полуфабрикаты': 'Другое', 'лед': 'Другое',
                    'овощи, грибы замороженные': 'плодоовощная продукция',
                    'кофе': 'кофе', 'чай': 'чай', 'другие горячие напитки': 'напитки безалкогольные',
                    'вода': 'напитки безалкогольные',
                    'газированные напитки, квас': 'напитки безалкогольные',
                    'соки, нектары, морсы': 'напитки безалкогольные',
                    'другие напитки': 'напитки безалкогольные', 'энергетические напитки': 'напитки безалкогольные',
                    'крупы': 'крупа и бобовые', 'макаронные изделия': 'макаронные изделия',
                    'бобовые': 'крупа и бобовые',
                    'хлопья, сухие завтраки': 'кондитерские изделия', 'мука': 'мука пшеничная',
                    'продукты быстрого приготовления': 'прочие продовольственные товары',
                    'сахар, соль, специи': 'Другое',
                    'товары восточной кухни': 'прочие продовольственные товары', 'масло, уксус': 'Другое',
                    'кетчупы, соусы': 'прочие продовольственные товары',
                    'овощи, грибы консервированные': 'плодоовощная продукция',
                    'сладкая консервация': 'плодоовощная продукция',
                    'готовые салаты, полуфабрикаты': 'плодоовощная продукция',
                    'чипсы, снеки': 'прочие продовольственные товары',
                    'орехи, сухофрукты, семечки': 'плодоовощная продукция', 'торты, бисквиты': 'кондитерские изделия',
                    'печенье, вафли, пряники': 'кондитерские изделия', 'шоколад, конфеты': 'кондитерские изделия',
                    'шоколадные и ореховые пасты': 'кондитерские изделия', 'зефир, мармелад': 'кондитерские изделия',
                    'восточные сладости': 'кондитерские изделия', 'жевательные резинки': 'кондитерские изделия',
                    'для вегетарианцев': 'Другое',
                    'хлебцы, клетчатка': 'Другое', 'продукты без глютена': 'Другое',
                    'без добавления сахара': 'кондитерские изделия',
                    'спортивное питание, бады': 'прочие продовольственные товары',
                    'детская гигиена': 'товары бытовой химии',
                    'смеси, каши': 'Другое',
                    'консервы, пюре': 'плодоовощная продукция', 'вода, соки, напитки': 'напитки безалкогольные',
                    'бакалея детская': 'прочие продовольственные товары',
                    'корм для собак и кошек': 'прочие культурнобытовые товары',
                    'гигиена и уход': 'товары бытовой химии',
                    'товары для птиц и грызунов': 'товары для спорта и туризма',
                    'средства для стирки': 'товары бытовой химии',
                    'средства для уборки': 'товары бытовой химии',
                    'средства для кухни и ванной': 'товары бытовой химии',
                    'средства по уходу за одеждой и обувью': 'бытовая химия',
                    'хозтовары для кухни': 'товары хозяйственные',
                    'хозтовары для дома': 'товары хозяйственные', 'уход за волосами': 'товары парфюмерно-косметические',
                    'уход за телом': 'товары парфюмерно-косметические',
                    'товары для бритья и депиляции': 'товары парфюмерно-косметические',
                    'гигиенические средства': 'товары бытовой химии',
                    'уход за лицом': 'товары парфюмерно-косметические', 'гигиена полости рта': 'товары бытовой химии',
                    'наборы косметики': 'товары парфюмерно-косметические',
                    'парфюмерия': 'товары парфюмерно-косметические',
                    'товары медицинского назначения': 'прочие товары',
                    'посуда, кухонные инструменты': 'посуда, приборы столовые', 'одежда и обувь': 'одежда',
                    'сад, огород': 'прочие товары', 'отдых на природе': 'товары для спорта и туризма',
                    'техника и электроника для дома': 'электротовары',
                    'канцелярские товары': 'письменные товары, канцелярские принадлежности',
                    'товары для творчества': 'письменные товары, канцелярские принадлежности',
                    'карты лояльности, сувениры': 'Другое',
                    'автотовары': 'прочие товары'}

In [103]:
df["category"] = df["category"].str.lower()
df['belstat_category'] = df['category'].map(category_mapping)
df

,Unnamed: 0,name,price,category,belstat_category
0,0,"Капуста белокочанная, 1 кг",1.35,овощи,плодоовощная продукция
1,1,"Картофель продовольственный, мытый, фасованный...",2.79,овощи,плодоовощная продукция
2,2,Картофель мытый,2.49,овощи,плодоовощная продукция
3,3,"Огурец гладкий, 1 кг",8.09,овощи,плодоовощная продукция
4,4,"Картофель, 1 кг",1.10,овощи,плодоовощная продукция
...,...,...,...,...,...
10086,10952,"Лопата для снега «Инструм-Агро» Снежок, планка...",16.08,автотовары,прочие товары
10087,10953,"Антифриз «Onzoil» Optimal, G 11, 900 мл",9.95,автотовары,прочие товары
10088,10954,"Буксировочный трос «Felix» 5 т, 5 м",23.05,автотовары,прочие товары
10089,10955,"Пуско-зарядное устройство «AVS» Energy BC-400,...",37.99,автотовары,прочие товары


In [104]:
df.loc[df["belstat_category"] == "Другое"]

,Unnamed: 0,name,price,category,belstat_category
3064,3394,"Пельмени «Мясные Подушечки» Боярские, 430 г",4.49,замороженные полуфабрикаты,Другое
3065,3395,"Пельмени «Мясные Подушечки» Домашние, 430 г",5.40,замороженные полуфабрикаты,Другое
3066,3396,"Тесто слоеное «Бабушка Аня» бездрожжевое, замо...",4.69,замороженные полуфабрикаты,Другое
3067,3397,"Блинчики «Бабушка Аня» с творогом, 360 г",4.57,замороженные полуфабрикаты,Другое
3068,3398,"Пельмени «Мясные Подушечки» Потешки, 430 г",5.07,замороженные полуфабрикаты,Другое
...,...,...,...,...,...
7754,8558,"Смесь молочная сухая «Nutrilak» Premium, 350 г",18.95,"смеси, каши",Другое
7755,8559,Каша гречневая «Kabrita» молочная с козьим мол...,23.35,"смеси, каши",Другое
7756,8560,"Кашагречневая «Kabrita» молочная, яблоко и абр...",25.00,"смеси, каши",Другое
7757,8561,"Смесь молочная сухая «HiPP» 3 Combiotic, 600 г",65.72,"смеси, каши",Другое


In [105]:
def classify_ambiguous_category(name, category):
    if "пельмени, вареники, равиоли" in category:
        if any(word in name.lower() for word in ["мяс", "говя", "свин", "индей"]):
            return "мясо и мясопродукты"
        elif any(word in name.lower() for word in ["карто", "капу", "гриб", "виш", 'шпин']):
            return "плодоовощная продукция"
        elif any(word in name.lower() for word in ["твор"]):
            return "молоко и молочные продукты"

    if "мясо, рыба" in category:
        if any(word in name.lower() for word in ["мяс", "говя", "свин", "индей", "бекон", "крол"]):
            return "мясо и мясопродукты"
        elif any(word in name.lower() for word in ["рыб", "минта"]):
            return "рыба и рыбопродукты"

    if "полуфабрикаты из свежего мяса, птицы и рыбы" in category:
        if any(word in name.lower() for word in
               ["мяс", "говя", "свин", "индей", "бекон", "крол", "цып", "ребр", "птиц", "колбас", "голуб"]):
            return "мясо и мясопродукты"
        elif any(word in name.lower() for word in ["рыб", "минта", "скумб"]):
            return "рыба и рыбопродукты"
        return "мясо и мясопродукты"

    if "полуфабрикаты из свежих овощей, тесто" in category:
        if any(word in name.lower() for word in ["тест"]):
            return "хлеб и изделия хлебобулочные"
        return "плодоовощная продукция"

    if "игрушки, творчество, развлечения" in category:
        if any(word in name.lower() for word in ["книг"]):
            return "печатные издания"
        return "прочие культурнобытовые товары"

    if "здоровое питание" in category:
        if any(word in name.lower() for word in ["хлеб"]):
            return "хлеб и изделия хлебобулочные"
        elif any(word in name.lower() for word in ["конф", "батон", "снек"]):
            return "кондитерские изделия"
        return "плодоовощная продукция"

    if "чай, кофе, какао" in category:
        if any(word in name.lower() for word in ["чай"]):
            return "чай"
        elif any(word in name.lower() for word in ["коф"]):
            return "кофе"
        return "кондитерские изделия"

    if "замороженные продукты" in category:
        if any(word in name.lower() for word in ["мяс", "говя", "свин", "индей"]):
            return "мясо и мясопродукты"
        elif any(word in name.lower() for word in ["карто", "капу", "гриб", "виш", 'шпин', 'смес']):
            return "плодоовощная продукция"
        elif any(word in name.lower() for word in ["твор"]):
            return "молоко и молочные продукты"
        elif any(word in name.lower() for word in ["тест"]):
            return "хлеб и изделия хлебобулочные"

    if "полуфабрикаты" in category:
        if any(word in name.lower() for word in ["мяс", "говя", "свин", "индей", "бекон", "крол",
                                                 "цып", "ребр", "птиц", "колбас", "голуб", "фрик",
                                                 "пельм"]):
            return "мясо и мясопродукты"
        elif any(word in name.lower() for word in ["рыб", "минта", "скумб"]):
            return "рыба и рыбопродукты"
        elif any(word in name.lower() for word in ["карто", "капу", "гриб", "виш", 'шпин', "малин"]):
            return "плодоовощная продукция"
        return "плодоовощная продукция"

    if "сахар, соль, специи" in category:
        if any(word in name.lower() for word in ["сахар", "подсла", "фрукт"]):
            return "сахар"
        elif any(word in name.lower() for word in ["пищ", "сол"]):
            return "соль поваренная пищевая"
        elif any(word in name.lower() for word in ["космет"]):
            return "товары парфюмерно-косметические"
        return "плодоовощная продукция"

    if "консервация мясная и рыбная" in category:
        if any(word in name.lower() for word in ["мяс", "говя", "свин", "индей", "бекон", "крол"]):
            return "мясо и мясопродукты"
        elif any(word in name.lower() for word in ["рыб", "минта", "лосос", "макрел", "тунец", "шпрот"]):
            return "рыба и рыбопродукты"
        return "мясо и мясопродукты"

    if "чипсы, снеки, сухарики" in category:
        if any(word in name.lower() for word in ["карто", "чипс", "солом"]):
            return "плодоовощная продукция"
        elif any(word in name.lower() for word in ["кур"]):
            return "мясо и мясопродукты"
        elif any(word in name.lower() for word in ["сухар", "гренк", "хлеб"]):
            return "хлеб и изделия хлебобулочные"
        elif any(word in name.lower() for word in ["анч"]):
            return "рыба и рыбопродукты"
        return "крупа и бобовые"

    if "блинчики, вареники, равиоли" in category:
        if any(word in name.lower() for word in ["мяс", "говя", "свин", "индей", "бекон", "крол",
                                                 "цып", "ребр", "птиц", "колбас", "голуб", "фрик",
                                                 "пельм"]):
            return "мясо и мясопродукты"
        elif any(word in name.lower() for word in ["рыб", "минта", "лосос", "макрел", "тунец", "шпрот"]):
            return "рыба и рыбопродукты"
        elif any(word in name.lower() for word in ["карто", "капу", "гриб", "виш", 'шпин', "малин"]):
            return "плодоовощная продукция"
        return "плодоовощная продукция"
    if "масло, уксус" in category:
        if any(word in name.lower() for word in ["масло"]):
            return "масло и жиры"
        else:
            return "прочие продовольственные товары"
    if "смеси, каши" in category:
        if any(word in name.lower() for word in ["каша"]):
            return "крупа и бобовые"
        if any(word in name.lower() for word in ["напиток"]):
            return "напитки безалкогольные"
        else:
            return "прочие продовольственные товары"

    if "хлебцы, клетчатка" in category:
        if any(word in name.lower() for word in ["хлебцы", "палочки", "полоски", "клетчатка"]):
            return "хлеб и изделия хлебобулочные"
        if any(word in name.lower() for word in ["чипсы", "корнерсы", "семена", "криспы"]):
            return "плодоовощная продукция"
        else:
            return "прочие продовольственные товары"

    if "продукты без глютена":
        if any(word in name.lower() for word in ["макар"]):
            return "макаронные изделия"
        if any(word in name.lower() for word in ["печенье", "хлопья", "батон", ""]):
            return "кондитерские изделия"
        else:
            return "прочие продовольственные товары"

    return "Другое"

In [106]:
df['belstat_category'] = df.apply(
    lambda row: classify_ambiguous_category(row['name'], row['category'])
    if row['belstat_category'] == "Другое"
    else row['belstat_category'],
    axis=1
)

In [107]:
df.loc[df["belstat_category"] == "Другое"]

,Unnamed: 0,name,price,category,belstat_category


In [108]:
df = df.loc[df["category"] != "лед"]

In [109]:
df.loc[df["belstat_category"] == "Другое"]

,Unnamed: 0,name,price,category,belstat_category


In [110]:
df.loc[df["belstat_category"] == "Другое"]["category"].unique()

array([], dtype=object)

In [111]:
df = df[(df["category"] != "карты лояльности, сувениры") & (df["category"] != "для вегетарианцев")]

In [112]:
df["belstat_category"].nunique()

30

In [113]:
df.groupby("belstat_category")["name"].count()

belstat_category
бытовая химия                                       31
кондитерские изделия                              1244
кофе                                               232
крупа и бобовые                                    216
макаронные изделия                                 179
масло и жиры                                       146
молоко и молочные продукты                         804
мороженое                                          104
мука пшеничная                                      54
мясо и мясопродукты                                793
напитки безалкогольные                             627
одежда                                              57
письменные товары, канцелярские принадлежности      69
плодоовощная продукция                            1446
посуда, приборы столовые                            72
прочие культурнобытовые товары                     201
прочие продовольственные товары                    705
прочие товары                                   

## алгоритм card

In [114]:
Z = df[['name', 'category']]
y = df['belstat_category']
X = Z['name'] + " " + Z['category']

vectorizer = TfidfVectorizer()
X_vectorized = vectorizer.fit_transform(X)

In [115]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

label_encoder = LabelEncoder()
y_label = label_encoder.fit_transform(y)

In [116]:
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [10, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

In [117]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y_label, test_size=0.2, random_state=42)
tree = DecisionTreeClassifier(random_state=42)
grid_search = GridSearchCV(estimator=tree, param_grid=param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)

grid_search.fit(X_train, y_train)

print(f"Лучшие параметры: {grid_search.best_params_}")

best_model = grid_search.best_estimator_
accuracy = best_model.score(X_test, y_test)
print(f"Точность модели с лучшими параметрами: {accuracy:.2f}")

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Лучшие параметры: {'criterion': 'entropy', 'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 2}
Точность модели с лучшими параметрами: 0.98


In [118]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

param_grid = {
    'C': [0.01, 0.1, 1, 10, 100]
}

svm = SVC(kernel='linear', random_state=42)

grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

print("Лучшие параметры:", grid_search.best_params_)

best_svm_linear = grid_search.best_estimator_
y_pred = best_svm_linear.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Лучшие параметры: {'C': 10}
Accuracy: 0.9970134395221503


In [119]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.01, 0.1, 1]
}

svm = SVC(kernel='rbf', random_state=42)
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

print("Лучшие параметры:", grid_search.best_params_)

best_svm_rbf = grid_search.best_estimator_
y_pred = best_svm_rbf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Лучшие параметры: {'C': 100, 'gamma': 0.1}
Accuracy: 0.9975111996017919


## Проверка на новых данных

In [126]:
df1 = pd.read_csv("/Users/denisovamaria/PycharmProjects/evroopt_data_preparing/evroopt_data_02.04.csv")

In [127]:
Z = df1[['name', 'category']]
X = Z['name'] + " " + Z['category']

X_vectorized = vectorizer.transform(X)

In [128]:
y_pred_evroopt_linear = best_svm_linear.predict(X_vectorized)

In [129]:
y_evroopt = label_encoder.inverse_transform(y_pred_evroopt_linear)

In [130]:
df1["belstat_category"] = y_evroopt

In [131]:
df1

,Unnamed: 0,name,price,category,number,belstat_category
0,0,"Лук репчатый, 1 кг",1.88,Овощи,720961,плодоовощная продукция
1,1,"Картофель продовольственный, мытый, фасованный...",2.89,Овощи,987256,плодоовощная продукция
2,2,"Огурец гладкий, 1 кг",7.10,Овощи,1348621,плодоовощная продукция
3,3,"Морковь, 1 кг",1.19,Овощи,428336,плодоовощная продукция
4,4,Свекла 1 кг,1.25,Овощи,2031147,плодоовощная продукция
...,...,...,...,...,...,...
10480,11412,"Набор кистей «ArtSpace» Пони №2,3,5",9.59,Товары для творчества,564868,"письменные товары, канцелярские принадлежности"
10481,11413,Стакан-непроливайка «Гамма» 2305215,1.69,Товары для творчества,1412725,"письменные товары, канцелярские принадлежности"
10482,11414,"Палитра для красок «Гамма» 10 ячеек, 2305212",3.25,Товары для творчества,1491050,"письменные товары, канцелярские принадлежности"
10483,11415,"Мелки восковые «Луч» Zoo, 12С865-08, 12 цветов",4.56,Товары для творчества,201133,"письменные товары, канцелярские принадлежности"


In [132]:
y_pred_evroopt_rbf = best_svm_rbf.predict(X_vectorized)
y_evroopt = label_encoder.inverse_transform(y_pred_evroopt_rbf)
df1["belstat_category"] = y_evroopt

In [134]:
df1

,Unnamed: 0,name,price,category,number,belstat_category
0,0,"Лук репчатый, 1 кг",1.88,Овощи,720961,плодоовощная продукция
1,1,"Картофель продовольственный, мытый, фасованный...",2.89,Овощи,987256,плодоовощная продукция
2,2,"Огурец гладкий, 1 кг",7.10,Овощи,1348621,плодоовощная продукция
3,3,"Морковь, 1 кг",1.19,Овощи,428336,плодоовощная продукция
4,4,Свекла 1 кг,1.25,Овощи,2031147,плодоовощная продукция
...,...,...,...,...,...,...
10480,11412,"Набор кистей «ArtSpace» Пони №2,3,5",9.59,Товары для творчества,564868,"письменные товары, канцелярские принадлежности"
10481,11413,Стакан-непроливайка «Гамма» 2305215,1.69,Товары для творчества,1412725,"письменные товары, канцелярские принадлежности"
10482,11414,"Палитра для красок «Гамма» 10 ячеек, 2305212",3.25,Товары для творчества,1491050,"письменные товары, канцелярские принадлежности"
10483,11415,"Мелки восковые «Луч» Zoo, 12С865-08, 12 цветов",4.56,Товары для творчества,201133,"письменные товары, канцелярские принадлежности"
